# 1. Basic Environment Setup 
- Include libraries / packages
- Manage Spark Session
- Import data with basic preview

In [1]:
# Import packages
import os
from datetime import datetime
from dateutil import tz
from pyspark.sql import SparkSession, SQLContext, Window
from pyspark.sql.functions import col, count, hour, isnan, json_tuple, lpad, \
                                  max, rank, udf, when
from pyspark.sql.types import StringType

In [2]:
# Setup spark session
spark = SparkSession.builder \
      .master('local') \
      .getOrCreate()

sqlContext = SQLContext(spark)

In [3]:
# Read snappy downloaded locally
df = sqlContext.read.parquet("part0.parquet")

df = df.union(sqlContext.read.parquet("part1.parquet")) \
  .union(sqlContext.read.parquet("part2.parquet")) \
  .union(sqlContext.read.parquet("part3.parquet")) 

In [4]:
# View basic data behavior for validation
print("Number of data:\n{0}\n".format(df.count()))
print("Data Schema:")
print(df.printSchema())

Number of data:
1303198

Data Schema:
root
 |-- server_ts: long (nullable = true)
 |-- e_n: string (nullable = true)
 |-- uid: string (nullable = true)
 |-- device: string (nullable = true)

None


In [5]:
# View first few lines of data to get some feeling...
df.show(5)

+-------------+------------------+--------+-------+
|    server_ts|               e_n|     uid| device|
+-------------+------------------+--------+-------+
|1526835750863| {"article_id":57}|e9b0971f| mobile|
|1526835751136|{"article_id":849}|b808aba2| mobile|
|1526835751188|{"article_id":871}|27f07360| mobile|
|1526835751215|{"article_id":150}|2abe5349|desktop|
|1526835751211|{"article_id":806}|338d71ea|desktop|
+-------------+------------------+--------+-------+
only showing top 5 rows



# 2. Data Validation and Cleaning (if needed)

## Validation
- Check for any null value
- Verifiy number of unique values in "device"

## Cleansing
- Clean any row with null value (?)
- Change server_ts: long -> datetime
- Change en: json_string -> string

In [6]:
# Data validation on null value
df.select([
    count(when(col(c).isNull(), c)).alias(c)
    for c in df.columns
]).show()

+---------+---+---+------+
|server_ts|e_n|uid|device|
+---------+---+---+------+
|        0|  0|119|     0|
+---------+---+---+------+



In [7]:
"""
uid empty sounds be acceptable anyway?...

Preparing cleaning method but optional...
"""
df = df.filter(df['uid'].isNotNull())
df.select([
    count(when(col(c).isNull(), c)).alias(c)
    for c in df.columns
]).show()

+---------+---+---+------+
|server_ts|e_n|uid|device|
+---------+---+---+------+
|        0|  0|  0|     0|
+---------+---+---+------+



In [8]:
# Validating distinct value of devices...
df.select('device').distinct().show()

+-------+
| device|
+-------+
|desktop|
| mobile|
+-------+



In [9]:
# Change server time
gmt_timezone = tz.gettz('GMT')
hk_timezone = tz.gettz('Asia/Hong_Kong')

def long_to_datetime(time_long):
    return datetime.fromtimestamp(float(time_long)/1000) \
      .replace(tzinfo=hk_timezone) \
      .astimezone(gmt_timezone) \
      .strftime('%Y-%m-%d %H:%M:%S.%f')

# Validate if function work as expected!
long_to_datetime(1526835750863)

'2018-05-20 17:02:30.863000'

In [10]:
# Apply long_to_datetime as a UDF...
long_to_datetime_udf = udf(long_to_datetime, StringType())
df = df.withColumn(
    'server_dt', 
    long_to_datetime_udf(df['server_ts'])
)

In [11]:
# Apply json_tuple in e_n. json_tuple is useful!
df = df.withColumn(
    'article_id',
    json_tuple(df['e_n'], 'article_id')
)

In [12]:
# Double Check if any null values for article_id...
df.select([
    count(when(col('article_id').isNull(), 'article_id'))
]).show()

+---------------------------------------------------------+
|count(CASE WHEN (article_id IS NULL) THEN article_id END)|
+---------------------------------------------------------+
|                                                       31|
+---------------------------------------------------------+



In [13]:
# Ouch ...
df.filter(df['article_id'].isNull()).show(5)

+-------------+---+--------+------+--------------------+----------+
|    server_ts|e_n|     uid|device|           server_dt|article_id|
+-------------+---+--------+------+--------------------+----------+
|1526829577338| {}|74143400|mobile|2018-05-20 15:19:...|      null|
|1526837193820| {}|577cbbc9|mobile|2018-05-20 17:26:...|      null|
|1526823633405| {}|656e375c|mobile|2018-05-20 13:40:...|      null|
|1526824766497| {}|7c0a3a8b|mobile|2018-05-20 13:59:...|      null|
|1526830116875| {}|69b067fe|mobile|2018-05-20 15:28:...|      null|
+-------------+---+--------+------+--------------------+----------+
only showing top 5 rows



In [14]:
# Better remvoe again
df = df.filter(df['article_id'].isNotNull())
df.filter(df['article_id'].isNull()).show(5)

+---------+---+---+------+---------+----------+
|server_ts|e_n|uid|device|server_dt|article_id|
+---------+---+---+------+---------+----------+
+---------+---+---+------+---------+----------+



### 3. Basic View Analysis
1. Check total numbers of events
2. Check best performed article...
3. Check best performed article within each hour...
4. Check avg / median for users

In [66]:
# 3.1 Check total (After clean)
df.count()

1303048

3.1: 
- Total numbers of event in this day: 1303048

In [16]:
# 3.2 Check best performed article
df.groupBy('article_id').count().orderBy('count', ascending=False).show(1)

+----------+-----+
|article_id|count|
+----------+-----+
|       827|49249|
+----------+-----+
only showing top 1 row



3.2: 
- Article with the greatest number of page views: 827
- Corresponding count: 49249   

In [17]:
# 3.3 Extract Hour
df = df.withColumn(
    'hour',
    lpad(hour(df['server_dt']), 2, '0')
)

In [67]:
# Double Group by as preparation ...
grouped_df = df.groupBy('hour', 'article_id').count()

# Setup partitionBy-window ...
hour_window = Window.partitionBy(grouped_df['hour']).orderBy(grouped_df['count'].desc())

# Rank over window and save as temporary column ...
grouped_df = grouped_df.withColumn(
     'rank',
    rank().over(hour_window)
)

# Filter and show
grouped_df.filter(grouped_df['rank'] == 1) \
      .select('hour', 'article_id', 'count') \
      .orderBy('hour').show(24)

+----+----------+-----+
|hour|article_id|count|
+----+----------+-----+
|  00|       830| 3105|
|  01|       155| 4933|
|  02|       830| 2914|
|  03|       374| 5404|
|  04|       374| 6139|
|  05|       897| 4418|
|  06|       813| 3214|
|  07|       740| 2517|
|  08|       712| 3045|
|  09|       712| 1937|
|  10|       827| 6702|
|  11|       827| 7198|
|  12|       827| 6428|
|  13|       827| 5518|
|  14|       827| 4806|
|  15|        67| 6942|
|  16|       827| 3139|
|  17|       871| 2274|
|  18|       827| 1072|
|  19|       827|  696|
|  20|       827|  391|
|  21|       827|  430|
|  22|        85| 1636|
|  23|        85| 4229|
+----+----------+-----+



3.3:
- Data table as show above

In [73]:
# 3.4 AVG / MEDIAN for user

# Sort for median
df_grp_by_uid = df.groupBy('uid').count().orderBy("count")

# Average
df_grp_by_uid.agg({
    'count': 'avg'
}).show()

# Approximate Median
df_grp_by_uid.approxQuantile("count", [0.5], 0.25)

+------------------+
|        avg(count)|
+------------------+
|2.3992826353942833|
+------------------+



[1.0]

3.4:
- Average: 2.400
- Median (Approximate): 1.0